# Group Project - Advanced Model

Divam Arora, Connor Moore, Hemanth Velan

DSBA 6165

### Sources:
* https://huggingface.co/datasets/gigaword
* https://huggingface.co/docs/datasets/process#export
* https://huggingface.co/docs/datasets/v1.11.0/splits.html
* https://www.geeksforgeeks.org/string-punctuation-in-python/
* https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
* https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html
* https://www.analyticsvidhya.com/blog/2021/06/pre-processing-of-text-data-in-nlp/
* https://stackoverflow.com/questions/42135409/removing-a-character-from-entire-data-frame
* https://discuss.huggingface.co/t/train-bart-for-conditional-generation-e-g-summarization/1904
* https://github.com/huggingface/transformers/blob/main/examples/pytorch/summarization/README.md
* https://www.analyticsvidhya.com/blog/2019/07/how-get-started-nlp-6-unique-ways-perform-tokenization/
* https://towardsdatascience.com/teaching-bart-to-rap-fine-tuning-hugging-faces-bart-model-41749d38f3ef
* https://stackoverflow.com/questions/28986489/how-to-replace-text-in-a-string-column-of-a-pandas-dataframe
* https://huggingface.co/docs/transformers/v4.35.0/en/model_doc/bart#transformers.BartForConditionalGeneration
* https://stackoverflow.com/questions/41425945/python-pandas-error-missing-unterminated-subpattern-at-position-2
* https://colab.research.google.com/drive/1Cy27V-7qqYatqMA7fEqG2kgMySZXw9I4?usp=sharing&pli=1#scrollTo=t77cjYY_fZlb
* https://aparnamishra144.medium.com/how-to-change-string-data-or-text-data-of-a-column-to-lowercase-in-pandas-248a8ce4ae01

First we need to re-run the code from our EDA/pre-processing notebook that loads and prepares our dataset for implementation.

In [24]:
# import needed packages
import re
import nltk
import time
import torch
import math
import random
import string
import evaluate
import argparse
import bert_score
import rouge_score
import numpy as np
import pandas as pd
import datasets as ds
from evaluate import load
import pytorch_lightning as pl
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from transformers import BartTokenizer, BartForConditionalGeneration

# download stop word package from nltk library
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cmoor197\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cmoor197\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cmoor197\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Dataset
Because our dataset is pulled directly from Huggingface's datasets library, there is no need for a local copy of the data. Running the cell below creates an instance of the specified dataset in your workspace environment.

In [2]:
# https://huggingface.co/datasets/gigaword
# https://huggingface.co/docs/datasets/v1.11.0/splits.html

# download gigaword dataset from Hugging Face dataset library
train, test, validation = ds.load_dataset("gigaword", split=["train", "test", "validation"])

In [3]:
# display the dataset splits
print(train)
print(test)
print(validation)

Dataset({
    features: ['document', 'summary'],
    num_rows: 3803957
})
Dataset({
    features: ['document', 'summary'],
    num_rows: 1951
})
Dataset({
    features: ['document', 'summary'],
    num_rows: 189651
})


In [4]:
# https://huggingface.co/docs/datasets/process#export

# export the training dataset to a pandas dataframe and display
df_train = train.to_pandas()
print("Train df exported.")

# export the test dataset to a pandas dataframe
df_test = test.to_pandas()
print("Test df exported.")

# export the validation dataset to a pandas dataframe
df_val = validation.to_pandas()
print("Validation df exported.")

Train df exported.
Test df exported.
Validation df exported.


### Balancing the train-test split
The standard provided division between train, test, and validation is extremely unbalanced towards train (95%), and the dataset overall is far too large to run through our model in a reasonable timespan. We decided to shrink the train set to 70,000 entries, and concat the provided test and validation sets. From that combined test-val set we will extract a 25,000-entry test set and a 5,000 entry validation set.

In [5]:
# select 70,000 rows randomly from the train dataframe

df_train_short = df_train.sample(n = 70000, random_state=2, ignore_index=True)

df_train_short

,document,summary
0,a british soldier was killed saturday by an ex...,british soldier killed in afghanistan blast
1,ukraine insists on building two new nuclear re...,ukraine insists on linking chernobyl closure t...
2,portuguese president mario soares will pay an ...,portugal 's president to visit angola next month
3,aol stepped up its transformation from interne...,aol introduces new advertising network plans t...
4,marine experts from wwf flew to the northern k...,suspected toxic algae bloom leaves thousands o...
...,...,...
69995,hong kong 's benchmark hang seng index ended h...,hong kong stocks edged up after four straight ...
69996,former brazil coach carlos alberto parreira sa...,parreira says he 's close to an agreement to c...
69997,around ## youths on thursday protested outside...,latvian youths protest ban of UNK symbols
69998,ohio 's method of putting prisoners to death i...,ohio judge says state s lethal injection proce...


In [6]:
# combine provided test and val sets and reseparate randomly into smaller subsets

# concat test and validation sets
test_val = [df_test, df_val]
df_testval_bulk = pd.concat(test_val)

# take a random sample of 30000 rows from the test and validation bulk set
df_testval_short = df_testval_bulk.sample(n = 30000, random_state=3, ignore_index=True)

# take a random 5000 row sample from the test-val subset
df_val_short = df_testval_short.sample(n = 5000, random_state=4, ignore_index=True)

# drop all rows taken for the validation sample from the test-val subset to create the test set
df_test_short = df_testval_short.drop(df_val_short.index, axis=0)

### Data Pre-Processing
We decided to truncate our pre-processing pipeline slightly from our baseline model notebook (removing lemmatization and not vectorizing text prior to passing it to the BART pretrained tokenizer) because BART models are designed to accept full, grammatically correct sentences. We thought passing more "normal" text during training may give the model better context and improve learning.

In [7]:
# the methods required to perform this function were found in this article -
# https://aparnamishra144.medium.com/how-to-change-string-data-or-text-data-of-a-column-to-lowercase-in-pandas-248a8ce4ae01
# the function and comments are our original work

# set all words in all rows to lower case

def lower(df):
    # vectorize strings in each row in summary column and set to lower case
    df["summary"] = df["summary"].str.lower()
    print("summary column lowercased")
    # vectorize strings in each row in document column and set to lower case
    df["document"] = df["document"].str.lower()
    print("document column lowercased")

In [8]:
# geeks for geeks and pandas doc pages were used as template source code and informed about parameter options
# stackoverflow posts helped with debugging issues
# https://stackoverflow.com/questions/42135409/removing-a-character-from-entire-data-frame
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html
# https://www.geeksforgeeks.org/string-punctuation-in-python/
# https://stackoverflow.com/questions/41425945/python-pandas-error-missing-unterminated-subpattern-at-position-2
# https://stackoverflow.com/questions/28986489/how-to-replace-text-in-a-string-column-of-a-pandas-dataframe
# comments and function are our original work, source code was modifed to fit our workspace

# remove all symbols and punctuation

# create instance of all punctuation symbols
punctuation = string.punctuation

# since we learned there are lots of apostrophe s in the dataset during EDA, we will add this to our remove list
punct_list = ["'s"]

# add all punctuation from the premade variable to our new list
for symbol in punctuation:
    punct_list.append(symbol)

# display the symbols included in our list
print(punct_list)

def remove_punctuation(df):
    # for each symbol in our punctuation list
    for symbol in punct_list:
        # iterate through the dataframe and replace every instance of the symbol with an empty string
        df["document"] = df["document"].str.replace(symbol, "", regex=False)
        df["summary"] = df["summary"].str.replace(symbol, "", regex=False)
    print("symbols removed")

["'s", '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']


In [9]:
# create data pre-processing pipeline

def pre_proc(df):
    # lowercase
    lower(df)
    # remove punctuation and symbols
    remove_punctuation(df)
    print("pre-processed successfully")

In [10]:
# call the data pre-processing pipeline for each of the dataset splits
# minus the lemmatization and tokenizer steps from the baseline notebook

pre_proc(df_train_short)
print("train df completed")
pre_proc(df_val_short)
print("test df completed")
pre_proc(df_test_short)
print("validation df completed")

# display new format of data using training set
df_train_short.head()

summary column lowercased
document column lowercased
symbols removed
pre-processed successfully
train df completed
summary column lowercased
document column lowercased
symbols removed
pre-processed successfully
test df completed
summary column lowercased
document column lowercased
symbols removed
pre-processed successfully
validation df completed


,document,summary
0,a british soldier was killed saturday by an ex...,british soldier killed in afghanistan blast
1,ukraine insists on building two new nuclear re...,ukraine insists on linking chernobyl closure t...
2,portuguese president mario soares will pay an ...,portugal president to visit angola next month
3,aol stepped up its transformation from interne...,aol introduces new advertising network plans t...
4,marine experts from wwf flew to the northern k...,suspected toxic algae bloom leaves thousands o...


Our dataset splits are now pre-processed and ready for use with models.

# Advanced Model Choices
*(from canvas) "Write about how your advanced model is different from your baseline model. Why did you choose the model architecture? What evidence from the previous model milestone did you use to drive your decision making? Write at least 100 words."*

The main difference between our advanced model and the baseline model is that in this advanced approach we attempted to train and fine tune the BART-base model for our text summarization task, rather than just using the BART-base model in its original form as we did with our baseline model. After strategies for fine-tuning and specializing transformer models were discussed in class, we wanted to try this with our model. BART, as a pretrained model, is very effective at text summarization out of the box, but we thought fine-tuning our model could potentially provide significant performance gains and make the model better at text summarization specifically. After some research, we found that BERT models seem to have far more examples of fine-tuning and task-oriented specialization than BART models, but we wanted to keep the same core transformer architecture as the baseline model to provide the best possible performance comparison. We were eventually able to find an example of someone fine-tuning a BART model for a specific task (linked in the towards data science article below), so we used that article and its associated google colab notebook as inspiration for model training and fine-tuning.

A change from the baseline model to this one that was a direct result of baseline performance is the adjustment we made to our "runBART" prediction function. During our baseline model test, the model produced an extremely large number of NaN empty predictions, which skews the metric scores (in addition to being completely useless from an end user perspective). Using the latter three sources listed below and some testing, we were able to determine a hyperparameter setup for the BART model's encode() and generate() functions that eliminated this problem completely. This advanced model implements these changes, and in 25,000 predictions output zero null values.

# Sources -
* https://towardsdatascience.com/teaching-bart-to-rap-fine-tuning-hugging-faces-bart-model-41749d38f3ef
* https://colab.research.google.com/drive/1Cy27V-7qqYatqMA7fEqG2kgMySZXw9I4?usp=sharing&pli=1#scrollTo=t77cjYY_fZlb
* https://github.com/huggingface/transformers/blob/main/examples/pytorch/summarization/README.md
* https://huggingface.co/docs/transformers/v4.35.0/en/model_doc/bart#transformers.BartForConditionalGeneration
* https://discuss.huggingface.co/t/train-bart-for-conditional-generation-e-g-summarization/1904


# Train and fine-tune BART model for text summarization
We attempted to optimize and fine tune our BART model taking inspiration from this article - https://towardsdatascience.com/teaching-bart-to-rap-fine-tuning-hugging-faces-bart-model-41749d38f3ef

using this affiliated notebook as source code - https://colab.research.google.com/drive/1Cy27V-7qqYatqMA7fEqG2kgMySZXw9I4?usp=sharing&pli=1#scrollTo=t77cjYY_fZlb

All cells with that colab link as the first line comment are based on the source notebook, we modified the approach utilized in that notebook and article to fit our workspace and changed some of the parameters and functions for our application. Variations are described in the cells with comments

In [11]:
# https://colab.research.google.com/drive/1Cy27V-7qqYatqMA7fEqG2kgMySZXw9I4?usp=sharing&pli=1#scrollTo=t77cjYY_fZlb

# load BART base model and tokenizer

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base', add_prefix_space=True)

bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

In [12]:
# https://colab.research.google.com/drive/1Cy27V-7qqYatqMA7fEqG2kgMySZXw9I4?usp=sharing&pli=1#scrollTo=t77cjYY_fZlb

# these functions are direct from the source notebook, these functions are used for the model training process
# several of the functions included in the source notebook have been removed because they do not fit our task

def shift_tokens_right(input_ids, pad_token_id):
  """ Shift input ids one token to the right, and wrap the last non pad token (usually <eos>).
      This is taken directly from modeling_bart.py
  """
  prev_output_tokens = input_ids.clone()
  index_of_eos = (input_ids.ne(pad_token_id).sum(dim=1) - 1).unsqueeze(-1)
  prev_output_tokens[:, 0] = input_ids.gather(1, index_of_eos).squeeze()
  prev_output_tokens[:, 1:] = input_ids[:, :-1]
  return prev_output_tokens

def encode_sentences(tokenizer, source_sentences, target_sentences, max_length=32, pad_to_max_length=True, return_tensors="pt"):
  ''' Function that tokenizes a sentence 
      Args: tokenizer - the BART tokenizer; source and target sentences are the source and target sentences
      Returns: Dictionary with keys: input_ids, attention_mask, target_ids
  '''

  input_ids = []
  attention_masks = []
  target_ids = []
  tokenized_sentences = {}

  for sentence in source_sentences:
    encoded_dict = tokenizer(
          sentence,
          max_length=max_length,
          padding="max_length" if pad_to_max_length else None,
          truncation=True,
          return_tensors=return_tensors,
          add_prefix_space = True
      )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

  input_ids = torch.cat(input_ids, dim = 0)
  attention_masks = torch.cat(attention_masks, dim = 0)

  for sentence in target_sentences:
    encoded_dict = tokenizer(
          sentence,
          max_length=max_length,
          padding="max_length" if pad_to_max_length else None,
          truncation=True,
          return_tensors=return_tensors,
          add_prefix_space = True
      )

    target_ids.append(encoded_dict['input_ids'])

  target_ids = torch.cat(target_ids, dim = 0)
  

  batch = {
      "input_ids": input_ids,
      "attention_mask": attention_masks,
      "labels": target_ids,
  }

  return batch  

In [13]:
# https://colab.research.google.com/drive/1Cy27V-7qqYatqMA7fEqG2kgMySZXw9I4?usp=sharing&pli=1#scrollTo=t77cjYY_fZlb

# this cell is also from the source notebook, setting initial states for hyperparameters prior to training

# create a baseline for the hyperparameters dictionary to pass to the model for training
params = argparse.Namespace()

params.freeze_encoder = True
params.freeze_embeds = True
params.eval_beams = 4

In [14]:
# https://colab.research.google.com/drive/1Cy27V-7qqYatqMA7fEqG2kgMySZXw9I4?usp=sharing&pli=1#scrollTo=t77cjYY_fZlb

# this model class is from the source notebook. variables/parameters and some of the functions were modified to fit our workspace and model task (summarization)

class LitModel(pl.LightningModule):
  # Instantiate the model
  def __init__(self, learning_rate, tokenizer, model, params):
    super().__init__()
    self.tokenizer = tokenizer
    self.model = model
    self.learning_rate = learning_rate
    self.params = params

    if self.params.freeze_encoder:
      freeze_params(self.model.get_encoder())

    if self.params.freeze_embeds:
      self.freeze_embeds()
  
  def freeze_embeds(self):
    ''' freeze the positional embedding parameters of the model; adapted from finetune.py '''
    freeze_params(self.model.model.shared)
    for d in [self.model.model.encoder, self.model.model.decoder]:
      freeze_params(d.embed_positions)
      freeze_params(d.embed_tokens)

  # Do a forward pass through the model
  def forward(self, input_ids, **kwargs):
    return self.model(input_ids, **kwargs)
  
  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr = self.learning_rate)
    return optimizer

  def training_step(self, batch, batch_idx):
    # Load the data into variables
    src_ids, src_mask = batch[0], batch[1]
    tgt_ids = batch[2]
    # Shift the decoder tokens right (but NOT the tgt_ids)
    decoder_input_ids = shift_tokens_right(tgt_ids, tokenizer.pad_token_id)

    # Run the model and get the logits
    outputs = self(src_ids, attention_mask=src_mask, decoder_input_ids=decoder_input_ids, use_cache=False)
    lm_logits = outputs[0]
    # Create the loss function
    ce_loss_fct = torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
    # Calculate the loss on the un-shifted tokens
    loss = ce_loss_fct(lm_logits.view(-1, lm_logits.shape[-1]), tgt_ids.view(-1))

    return {'loss':loss}

  def validation_step(self, batch, batch_idx):

    src_ids, src_mask = batch[0], batch[1]
    tgt_ids = batch[2]

    decoder_input_ids = shift_tokens_right(tgt_ids, tokenizer.pad_token_id)
    
    # Run the model and get the logits
    outputs = self(src_ids, attention_mask=src_mask, decoder_input_ids=decoder_input_ids, use_cache=False)
    lm_logits = outputs[0]

    ce_loss_fct = torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
    val_loss = ce_loss_fct(lm_logits.view(-1, lm_logits.shape[-1]), tgt_ids.view(-1))

    return {'loss': val_loss}
  
  # this function was heavily modified from the source code to fit our application, utilizing the new configuration we found to eliminate NaN prediction outputs
  def generate_text(self, inputs, max_length, min_length, length_penalty, num_beams, early_stopping=True):
    ''' Function to generate text '''
    generated_id = self.model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=length_penalty, num_beams=num_beams, early_stopping=early_stopping)
    return generated_id

def freeze_params(model):
  ''' Function that takes a model as input (or part of a model) and freezes the layers for faster training
      adapted from finetune.py '''
  for layer in model.parameters():
    layer.requires_grade = False

In [15]:
# https://colab.research.google.com/drive/1Cy27V-7qqYatqMA7fEqG2kgMySZXw9I4?usp=sharing&pli=1#scrollTo=t77cjYY_fZlb

# this class is also from the source notebook. input parameters and all of the dataframe column name calls for the "setup" function had to be modified for our workspace

# Create a dataloading module as per the PyTorch Lightning Docs
class SummaryDataModule(pl.LightningDataModule):
  def __init__(self, tokenizer, train, test, validate, batch_size):
    super().__init__()
    self.tokenizer = tokenizer
    self.batch_size = batch_size
    self.train = train
    self.test = test
    self.validate = validate

  # encode the sentences using the tokenizer  
  def setup(self, stage):
    self.train = encode_sentences(self.tokenizer, self.train['document'], self.train['summary'])
    self.validate = encode_sentences(self.tokenizer, self.validate['document'], self.validate['summary'])
    self.test = encode_sentences(self.tokenizer, self.test['document'], self.test['summary'])

  # Load the training, validation and test sets in Pytorch Dataset objects
  def train_dataloader(self):
    dataset = TensorDataset(self.train['input_ids'], self.train['attention_mask'], self.train['labels'])                          
    train_data = DataLoader(dataset, num_workers=7, persistent_workers=True, sampler = RandomSampler(dataset), batch_size = self.batch_size)
    return train_data

  def val_dataloader(self):
    dataset = TensorDataset(self.validate['input_ids'], self.validate['attention_mask'], self.validate['labels']) 
    val_data = DataLoader(dataset, num_workers=7, persistent_workers=True, batch_size = self.batch_size)                       
    return val_data

  def test_dataloader(self):
    dataset = TensorDataset(self.test['input_ids'], self.test['attention_mask'], self.test['labels']) 
    test_data = DataLoader(dataset, num_workers=7, persistent_workers=True, batch_size = self.batch_size)                   
    return test_data

In [16]:
# https://colab.research.google.com/drive/1Cy27V-7qqYatqMA7fEqG2kgMySZXw9I4?usp=sharing&pli=1#scrollTo=t77cjYY_fZlb

# this cell is from the source notebook, we changed the input dataset variables

# create an instance of the data summary class using our datasets
summary_data = SummaryDataModule(tokenizer, df_train_short, df_test_short, df_val_short, batch_size = 16)

# create an instance of the model
model = LitModel(learning_rate=2e-5, tokenizer=tokenizer, model=bart_model, params=params)

In [17]:
# https://colab.research.google.com/drive/1Cy27V-7qqYatqMA7fEqG2kgMySZXw9I4?usp=sharing&pli=1#scrollTo=t77cjYY_fZlb

# this cell is from the source notebook, it was modified through trial and error to find the setup that worked best with our model

# create a training instance of our model, hyperparameters were modified from the source code
# we tried one epoch first because the source code used one epoch and was able to get good results and even training for only one epoch still took almost 4 hours
trainer = pl.Trainer(logger=False,
                     max_epochs = 1,
                     min_epochs = 1,
                     enable_model_summary=True,
                     enable_progress_bar=True
                     )

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [18]:
# https://colab.research.google.com/drive/1Cy27V-7qqYatqMA7fEqG2kgMySZXw9I4?usp=sharing&pli=1#scrollTo=t77cjYY_fZlb

# this is the last cell referencing the source notebook

# fit the instance of our model to the dataset instance
trainer.fit(model, summary_data)


  | Name  | Type                         | Params
-------------------------------------------------------
0 | model | BartForConditionalGeneration | 139 M 
-------------------------------------------------------
139 M     Trainable params
0         Non-trainable params
139 M     Total params
557.682   Total estimated model params size (MB)


Epoch 0: 100%|██████████| 4375/4375 [3:44:04<00:00,  0.33it/s]             

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 4375/4375 [3:44:13<00:00,  0.33it/s]


end of using that source notebook - trying out the trained model 

In [19]:
# new version OF runBart function, updated from our baseline model for improved speed and performance
# testing and trial and error showed that removing the component of the function that calculated a unique maxlength for every summary sped up prediction generation significantly
# in the baseline model we had issues with large numbers of NaN empty summary predictions,
# through some research, testing, and trial and error we were able to find a configuration for the encode and generate() steps that corrected this issue

def runBart(df):

    # Empty lists for predictions and performance timestamps
    predictions = []
    times = []

    # For the number of rows in the given dataframe
    for i in range(len(df)):
        # Create a start timestamp
        start = time.perf_counter()

        # Create a document instance using the row's entry for the stringified document
        doc = df.iloc[i]["document"]

        # Encoding inputs using BART tokenizer 
        inputs = tokenizer.encode(doc, return_tensors='pt', max_length=1024, truncation=True)

        # Generate vectorized summary using encoded inputs
        summary_ids = model.generate_text(inputs, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)

        # Decode the summary into a human-readable format
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        # Append the predicted summary to a list of predictions
        predictions.append(summary)

        # Create an end timestamp
        end = time.perf_counter()

        # Calculate computation speed
        speed = end - start

        # Append computation speed to list
        times.append(speed)

        # If the iteration is a multiple of 1000
        if i % 5000 == 0:
            # Calculate the average computation time per row so far and print
            avg_time = sum(times) / len(times)
            print("Average time per row at", i, "row:", avg_time)

    # Create a new column for the dataframe using the predictions generated and return the modified dataframe
    df["BART_Pred"] = predictions
    return df

In [20]:
# run the model to generate predictions using the test set
runBart(df_test_short)

df_test_short

Average time per row at 0 row: 2.058996300000217
Average time per row at 5000 row: 1.918337959508114
Average time per row at 10000 row: 1.9208283956404233
Average time per row at 15000 row: 1.9187569718818536
Average time per row at 20000 row: 1.9171497695365232


,document,summary,BART_Pred
5000,japanese electronics giant toshiba said tuesda...,toshiba profits times up in third quarter,toshiba net profits grow more than times yea...
5001,michael campbell opened a unk lead on defendin...,campbell puts defending champion woosnam in tr...,campbell leads woosnam after holes in quarte...
5002,iran on tuesday dismissed the allegation by th...,iran denies allegation on its military deployment,iran dismisses alawsat claim on southern bord...
5003,turkish foreign minister and deputy prime mini...,turkish fm hails eu plan to end economic sanct...,turkish fm says eu proposal on cypriot cyprio...
5004,patricia mcgovern the former senate ways and ...,former senator joins governor race,mcgovern announces campaign for governor of t...
...,...,...,...
29995,three activists are charged with staging an un...,charges against activists could set precedent ...,hong kong activists charged with unauthorized...
29996,a prominent fatah leader and his yearold son...,father son killed in fatahhamas fight in gaza,fatah leader hamas killed in gaza city fighti...
29997,trustees at virginia union university one of ...,wilder in running to head virginia union,virginia union trustees considering douglas w...
29998,patrouille de france lrb paf rrb the famous...,french famous unk to present aerobatics shows ...,china aerobatic team to present aerobatics s...


In [22]:
# check generated predictions for NaN values

# count null values in BART pred columnm
null_predictions = df_test_short['BART_Pred'].isna().sum()

print("there were", null_predictions, "empty predictions generated.")

there were 0 empty predictions generated.


This is an improvement over our baseline model, which generated a large number of null values as predictions.

# BERTScore Metrics

In [25]:
# initialize BERTScore metric

bertscore = load("bertscore")

In [26]:
# generating BERTScore metrics for model predictions

# create list of prediction outputs
test_predictions = list(df_test_short["BART_Pred"].astype(str))
# create list of true outputs
test_references = list(df_test_short["summary"].astype(str))
# calculate BERTScore values comparing model predictions with true summaries
test_results_bert = bertscore.compute(predictions=test_predictions, references=test_references, lang="en")

config.json: 100%|██████████| 482/482 [00:00<00:00, 161kB/s]
C:\Users\cmoor197\AppData\Roaming\Python\Python39\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cmoor197\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
vocab.json: 100%|██████████| 899k/899k [00:00<00:00, 11.2MB/s]
m

In [27]:
# calculate average precision, recall and F1 scores from BERTScore model based on model predictions

# create list of result keys
test_keys = list(test_results_bert.keys())

# for number of values in keylist-1
for k in range(len(test_keys)-1):
    # sum total of all result values
    s_test = sum(test_results_bert[test_keys[k]])
    # calculate the total number of result values
    le_test = len(test_results_bert[test_keys[k]])
    # compute average result value
    avg_test = s_test/le_test

    print("test set result:")
    print("Average {} is {}".format(test_keys[k], avg_test))

test set result:
Average precision is 0.8033035963749886
test set result:
Average recall is 0.8921198317599297
test set result:
Average f1 is 0.8449282884764672


# ROUGE Metrics

In [31]:
# initialize ROUGE metrics model

rouge = load('rouge')

In [33]:
# generate ROUGE metrics scores for train and test model outputs

# compute ROUGE metrics scores comparing model predictions with true outputs
test_results_rouge = rouge.compute(predictions=test_predictions, references=test_references)

print("test set results:")
print(test_results_rouge)

test set results:
{'rouge1': 0.20933892884082345, 'rouge2': 0.08089128435239316, 'rougeL': 0.18351244174179016, 'rougeLsum': 0.18348750351265208}


# Observe a few specific model predictions for human evaluation

We wanted to view a few of the model predictions beside their target outputs and the original inputs to see how the model performed by human standards, as metrics can only convey so much meaning in evaluating high-level NLP tasks by themselves.

In [45]:
# produce five random input/target/prediction pairs from the model

# empty list for index sample
indexList = []

# generate five random index values and add to list
for i in range(5):
    index = random.randint(0,25000)
    indexList.append(index)

print(indexList)
print()

# for index in list
for ind in indexList:
    # display the original input, the target summary, and what the model predicted for a summary
    print("index:", ind)
    print("original input:", df_test_short.iloc[ind]["document"])
    print("target summary:", df_test_short.iloc[ind]["summary"])
    print("model prediction:", df_test_short.iloc[ind]["BART_Pred"])
    print()


[13441, 705, 23703, 20834, 21140]

index: 13441
original input: steve mcmanaman might get back into the england team after all 
target summary: england offers hope for recall of real madrid star
model prediction:  steve mcmanaman might get back into england team after all that time has passed  mcmanmanaman may get back in the lineup after all this time he has been out of the netherlands with england this year 

index: 705
original input: the federal reserve bank of new york has taken disciplinary action against jp morgan amp co 
target summary: jp morgan disciplined for loans to sumitomo trader
model prediction:  federal bank of new york takes disciplinary action against jp morgan co  co  bank of cambodia bank says he  bank should not be allowed to do business in the state banking system for  years or face loss of millions

index: 23703
original input: china will turn all their resources to the th asian women  basketball championships  which will open on may  in japan  to earn the only

# Model Performance Results
*(from canvas) "How does your model perform on the metrics you have chosen from your previous submission?"*

As can be seen above, disappointingly, our model, after one epoch of fine-tuning/training, did not perform very well in comparison to our baseline model. BERTScore recall did increase by 3% (from 86 to 89), but all other metrics dropped by a few percentage points. However, it is worth noting that our baseline model produced an extremely large number of null values (and this one produced none !), which would skew the way these metrics would evaluate the model, so a direct comparison is not as meaningful as it could be. 

As was discussed in class, numeric n-gram based metrics like ROUGE can only be so valuable for complex NLP tasks like text summarization, as a model can semantically capture a target value without necessarily using the same words or grammar structure, so we are not as discouraged that these scores are lower than the baseline. The BERTScore value, however, is semantic, so this score decreasing is more disheartening. Something we did find interesting was actually reading the model outputs vs the target summaries vs the original inputs ourselves, as this seems to paint a better picture of where the model is in terms of learning and how it is performing. As can be seen in the examples above, the model is capable of producing (mostly) coherent summaries pretty consistently, so that is encouraging. Something we thought was interesting was the way that the model's summary predictions are often longer than the original input text. We realized, a little too late in the project process, that the dataset we chose for this project has entries to be summarized that are, on average, very short (typically no more than a few sentences). We believe this may have been preventative in the model's learning, and makes the task of summarization much more difficult for the model than it would be with a larger corpus of text (which provides more context, and more content for the model to condense into shorthand that makes sense). "Summarizing" one sentence in one sentence is difficult, even for a human. We have learned about text summarization throughout the semester, and have a much better understanding of the kind of data that is best suited for this task now than we did in August.

Another note based on reading the model's outputs - the model frequently fixated on a word or n-gram and repeated itself several times in a way that sort of "padded" summaries in garbage filler. We do believe that this behavior could be lessened with additional and more thorough model training. We only ran our training fit() process for one epoch, with 70,000 rows of training data, and this still took almost 4 hours to complete. This, when combined with the over 14 hours required to run the remaining components of the notebook, made for an over 18 hour process from start to finish. A model test run taking that long was extremely restrictive when it came to testing, debugging, and trial-and-error adjustments, so for the sake of turning the assignment in on time we had to settle for lower performance than we wanted. We are, however, going to continue running the model with additional epochs and more training data until the time of the final project presentation, just to see if we can improve model performance more. If we are able to get better numbers we will report these in our final report and presentation.